# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.preprocessing import QuantileTransformer
clf = QuantileTransformer(n_quantiles=10)
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


QuantileTransformer(n_quantiles=10)

In [6]:
clf.n_features_in_

4

## Deploying the Model

In [7]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [8]:
print(lCPPCode)

namespace  {

	const tAnyVector lRefQuantileData = { 0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.4444444444444444, 0.5555555555555556, 0.6666666666666666, 0.7777777777777777, 0.8888888888888888, 1.0 };
	

	const std::map<int, tAnyVector> lFeatureQuantile_data = {
			{ 0 ,  {4.3, 4.9, 5.1, 5.4, 5.700000000000001, 5.977777777777779, 6.3, 6.5, 6.844444444444443, 7.9 }} ,
			{ 1 ,  {2.0, 2.5, 2.8, 2.9, 3.0, 3.077777777777779, 3.2, 3.4, 3.6, 4.4 }} ,
			{ 2 ,  {1.0, 1.4, 1.5, 2.633333333333323, 4.1, 4.5, 4.9, 5.188888888888887, 5.7, 6.9 }} ,
			{ 3 ,  {0.1, 0.2, 0.21111111111111075, 0.8666666666666629, 1.3000000000000003, 1.4, 1.6, 1.888888888888887, 2.144444444444443, 2.5 }} 
	};
	

	std::vector<std::string> get_input_names(){
		std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };

		return lFeatures;
	}

	std::vector<std::string> get_output_names(){
		std::vector<std::string> lOutputs = { "Feature_0", "Feature_1", "Feature_2",

In [9]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [10]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_139955588534864.exe /tmp/sklearn2sql_cpp_139955588534864.cpp'
idx,Feature_0,Feature_1,Feature_2,Feature_3
0,0.22222222222222,0.83333333333333,0.11111111111111,0.11111111111111
1,0.11111111111111,0.44444444444444,0.11111111111111,0.11111111111111
2,0.07407407407407,0.66666666666667,0.08333333333333,0.11111111111111
3,0.05555555555556,0.57575757575758,0.22222222222222,0.11111111111111
4,0.16666666666667,0.88888888888889,0.11111111111111,0.11111111111111
5,0.33333333333333,0.93055555555556,0.24183006535948,0.25423728813559
6,0.05555555555556,0.77777777777778,0.11111111111111,0.23728813559322
7,0.16666666666667,0.77777777777778,0.22222222222222,0.11111111111111
8,0.01851851851852,0.33333333333333,0.11111111111111,0.11111111111111
9,0.11111111111111,0.57575757575758,0.22222222222222,0.00000000000000
10,0.33333333333333,0.90277777777778,0.22222222222222,0.11111111111111
11,0.09259259259259,0.77777777777778,0.2320

In [11]:
cpp_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,0.484444,0.222222,0.743590,0.968750
74,74,0.722222,0.333333,0.500000,0.444444
9,9,0.111111,0.575758,0.222222,0.000000
88,88,0.407407,0.444444,0.444444,0.444444
25,25,0.166667,0.444444,0.232026,0.111111
5,5,0.333333,0.930556,0.241830,0.254237
48,48,0.296296,0.902778,0.222222,0.111111
117,117,0.978947,0.916667,0.981481,0.906250
83,83,0.563218,0.185185,0.743590,0.666667
105,105,0.968421,0.444444,0.972222,0.869565


In [12]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_transform = pd.DataFrame(clf.transform(X), columns=cpp_output.columns[1:]);
skl_output = pd.concat([skl_output_key, skl_output_transform] , axis=1)
skl_output.sample(12, random_state=1960)

,idx,Feature_0,Feature_1,Feature_2,Feature_3
114,114,0.484444,0.222222,0.743590,0.968750
74,74,0.722222,0.333333,0.500000,0.444444
9,9,0.111111,0.575758,0.222222,0.000000
88,88,0.407407,0.444444,0.444444,0.444444
25,25,0.166667,0.444444,0.232026,0.111111
5,5,0.333333,0.930556,0.241830,0.254237
48,48,0.296296,0.902778,0.222222,0.111111
117,117,0.978947,0.916667,0.981481,0.906250
83,83,0.563218,0.185185,0.743590,0.666667
105,105,0.968421,0.444444,0.972222,0.869565


In [13]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [14]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Feature_0_skl,Feature_1_skl,Feature_2_skl,Feature_3_skl,idx_cpp,Feature_0_cpp,Feature_1_cpp,Feature_2_cpp,Feature_3_cpp
114,114,0.484444,0.222222,0.743590,0.968750,114,0.484444,0.222222,0.743590,0.968750
74,74,0.722222,0.333333,0.500000,0.444444,74,0.722222,0.333333,0.500000,0.444444
9,9,0.111111,0.575758,0.222222,0.000000,9,0.111111,0.575758,0.222222,0.000000
88,88,0.407407,0.444444,0.444444,0.444444,88,0.407407,0.444444,0.444444,0.444444
25,25,0.166667,0.444444,0.232026,0.111111,25,0.166667,0.444444,0.232026,0.111111
5,5,0.333333,0.930556,0.241830,0.254237,5,0.333333,0.930556,0.241830,0.254237
48,48,0.296296,0.902778,0.222222,0.111111,48,0.296296,0.902778,0.222222,0.111111
117,117,0.978947,0.916667,0.981481,0.906250,117,0.978947,0.916667,0.981481,0.906250
83,83,0.563218,0.185185,0.743590,0.666667,83,0.563218,0.185185,0.743590,0.666667
105,105,0.968421,0.444444,0.972222,0.869565,105,0.968421,0.444444,0.972222,0.869565


In [15]:
for col in cpp_output.columns:
    lDiff = cpp_skl_join[col + "_skl"] - cpp_skl_join[col + "_cpp"]
    print(lDiff.describe())

count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
dtype: float64
count    1.500000e+02
mean    -2.379347e-16
std      3.080697e-15
min     -4.996004e-15
25%     -3.094747e-15
50%     -3.330669e-16
75%      2.498002e-15
max      4.218847e-15
dtype: float64
count    1.500000e+02
mean     4.265107e-17
std      3.421721e-15
min     -4.884981e-15
25%     -3.372302e-15
50%      1.096345e-15
75%      3.330669e-15
max      4.482525e-15
dtype: float64
count    1.500000e+02
mean    -1.043610e-16
std      2.883687e-15
min     -4.662937e-15
25%     -2.976785e-15
50%      0.000000e+00
75%      2.192690e-15
max      4.440892e-15
dtype: float64
count    1.500000e+02
mean     1.083115e-15
std      2.557713e-15
min     -4.551914e-15
25%      0.000000e+00
50%      1.096345e-15
75%      3.552714e-15
max      4.884981e-15
dtype: float64
